In [9]:
import sys
sys.path.append('..')

import os
from dotenv import load_dotenv
import pandas as pd
from tqdm.notebook import tqdm

from src.spotify import Spotify
import src.file_utils 

spot = Spotify()

In [4]:
# testing spotify api
test_song = 'https://open.spotify.com/track/7CRPXJt3IVC7V0VcM0PSQi?si=79e0c2a7a20d49eb'

info = spot.get_track_info(test_song)
print(info)

KeyboardInterrupt: 

In [22]:
# directory of original json pages
JSON_PATH = '../data/chartex_json'

# directory of csv pages
CSV_PATH = '../data/chartex_csv'

# filepath of merged raw data
MERGED_PATH = '../data/chartex_merged.csv'

# file path of data with only relevant columns
CLEAN_PATH = '../data/chartex_clean.csv'

# filepath of data with no duplicate songs
FINAL_PATH = '../data/chartex_final.csv'

### Get data from spotify and save to csv

In [18]:
def chartex_to_csv(page_path, out_path):
    '''add spotify data to chartex data and save to csv'''
    page_data = file_utils.load_json(page_path)
    assert(page_data)
    tracks = []
    for chartex_track_info in tqdm(page_data['results'], desc=f'tracks'):
        spoitfy_id = chartex_track_info['spotify_platform_id']
        if not spoitfy_id:
            continue
        track_info = spot.get_track_info(spoitfy_id)
        track_info.update(chartex_track_info)
        tracks.append(track_info)
        
    raw_df = pd.DataFrame(tracks)
    raw_df.to_csv(out_path, index=False)

# getting spotify data for all pages
for page in tqdm(range(18,31), desc='pages'):
    json_path = f'{JSON_PATH}/page{page}.json'
    csv_path = f'{CSV_PATH}/page{page}.csv'
    chartex_to_csv(json_path, csv_path)


pages:   0%|          | 0/13 [00:00<?, ?it/s]

tracks:   0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [24]:
#combining all csv files into one
file_utils.merge_csv(CSV_PATH, MERGED_PATH)

### cleaning up the data


In [34]:
#combining all csv files into one
merged_path = '../data/chartex_merged.csv'
#src.file_utils.merge_csv(OUTPATH, merged_path)
src.file_utils.merge_csv(OUTPATH, merged_path)

Total num rows: 4447
Num rows after dropping rows with no id: 4443
Num rows after dropping rows with no audio features: 4443


In [8]:
# remove unnecessary columns 
to_drop = ["type", "uri", "track_uri", "track_href", "rank", "youtube_platform_id", 
           "tiktok_platform_id", "itunes_platform_id", "spotify_platform_id" ,
           "itunes_album_platform_id", "song_guid", "analysis_url", "length"]
merged_df = pd.read_csv(MERGED_PATH)
clean_df = merged_df.drop(to_drop, axis=1)

# remove rows with no id
print(f'Total num rows: {clean_df.shape[0]}')
clean_df = clean_df.dropna(subset=['id'])
print(f'Num rows after dropping rows with no id: {clean_df.shape[0]}')

# remove rows with no audio features
clean_df = clean_df.dropna(subset=['valence'])
print(f'Num rows after dropping rows with no audio features: {clean_df.shape[0]}')

# save to csv
clean_df.to_csv(CLEAN_PATH, index=False)

Total num rows: 4447
Num rows after dropping rows with no id: 4443
Num rows after dropping rows with no audio features: 4443


### Merging duplicate tracks

In [47]:
# Load the clean CSV file into a DataFrame
clean_df = pd.read_csv(CLEAN_PATH)
print(f'The dataset has {clean_df.shape[0]} rows and {clean_df.shape[1]} columns.')

# Find how many tracks with identical names are in the dataset
duplicates = clean_df.duplicated(subset=['track_name'], keep=False)
num_duplicates = duplicates.sum()
print(f'There are {num_duplicates} duplicate tracks.') 

# example of duplicate tracks
# df_copy = clean_df.copy()
indices = clean_df.loc[clean_df['track_name'] == 'Laxed – Siren Beat'].index
example_duplicates = clean_df.loc[indices, ['track_name', 'number_of_videos']]
print(f'\nExample of duplicated tracks:\n{example_duplicates}\n')

# Group the DataFrame by 'track_name' and sum up the 'number_of_videos'
grouped_df = clean_df.groupby('track_name', as_index=False)
grouped_df = grouped_df.agg({col: 'first' if col != 'number_of_videos' else 'sum' for col in clean_df.columns})
grouped_df = grouped_df.reset_index()
print(f'without duplicates the DataFrame has {grouped_df.shape[0]} rows.')

# example of merged track
index = grouped_df.loc[grouped_df['track_name'] == 'Laxed – Siren Beat'].index
example_merged = grouped_df.loc[index, ['track_name', 'number_of_videos']]
print(f'\nExample of merged tracks:\n{example_merged}\n')

# sort by popularity
sorted_df = grouped_df.sort_values(by="number_of_videos", ascending=False)

# save to csv
sorted_df.to_csv(FINAL_PATH, index=False)

The dataset has 4443 rows and 24 columns.
There are 915 duplicate tracks.

Example of duplicated tracks:
              track_name  number_of_videos
78    Laxed – Siren Beat           1200000
462   Laxed – Siren Beat           1100000
1021  Laxed – Siren Beat            149300
2149  Laxed – Siren Beat          34700000
2249  Laxed – Siren Beat           3900000

without duplicates the DataFrame has 3932 rows.

Example of merged tracks:
              track_name  number_of_videos
1679  Laxed – Siren Beat          41049300

